In [111]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time 
import xl2dict
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件


# 找到excel資料存放的位置，將所有檔案的路徑變成list

In [113]:
myxlobject= xl2dict.XlToDict()
dataDict=myxlobject.convert_sheet_to_dict(file_path=pathList[0], sheet="First Sheet")
dataDict[0]

{'個人自願提繳率(%)': '',
 '勞基法特殊身分別': '',
 '勞工保險證號': '05952202',
 '勞工保險證號檢查碼': 'F',
 '勞退提繳日期(與勞保加保日期不同時才需輸入)': '',
 '已領取社會保險給付種類': '',
 '提繳身分別': '1',
 '月實際工資': '11100',
 '格式別': '1',
 '特殊身分別': '1',
 '異動別': '4',
 '被保險人出生日期': '0880224',
 '被保險人外籍': '',
 '被保險人姓名': '吳名豪',
 '被保險人性別': '',
 '被保險人身份證號': 'A130528512',
 '雇主提繳率(%)': '6'}

# 將每個excel轉成dict，並且判斷裡面內容:
### 　　單位-保險證號
### 　　加保、退保、調保
### 　　三合一、二合一

# 讓使用者輸入:
### 　　pin碼
### 　　檔案資料夾位置

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage



try:

    pinNum="780311" #需改成人工輸入

    chrome_options = Options()
    #crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
    chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
    #webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
    #套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
    driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
    driver.get("https://edesk.bli.gov.tw/cpa/")
    
    #找到excel資料存放的位置，將所有檔案的路徑變成list
    dataDir=r"D:\保險申報檔案" #需改成人工輸入
    #遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
    # for i in os.walk(dataDir):
    #     for j in i[2]:
    #         print(i[0]+j)
    #可縮寫成這樣     
    pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
    myxlobject= xl2dict.XlToDict()
    for excelFilePath in pathList:
        #得到excel內容成dict
        dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
        #進入勞保局
        WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
    #     alert=driver.switch_to_alert()
    #     alert.dismiss()
        #關閉阻擋頁
        driver.find_element_by_class_name("b-close").click()
        #輸入pin碼(自然人憑證)
        pinInput=driver.find_element_by_name("pin")
        pinInput.clear()
        pinInput.send_keys(pinNum)#憑證pin碼
        #輸入保險證號
        unoInput=driver.find_element_by_name("uno")
        unoInput.clear()
        unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
        time.sleep(0.5)
        #確認進入
        driver.find_element_by_name('Submit').click()
        WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
        passOrNot=True
        driver.find_element_by_class_name("b-close").click()
        time.sleep(0.4)
        driver.find_element_by_id("mi0,0").click()
        time.sleep(0.1)
        driver.find_element_by_id("mi0,5").click()
        time.sleep(0.2)

        if dataDict['異動別']=="4":
            driver.find_element_by_id("mi0,6").click()#加保
        elif dataDict['異動別']=="2":
            driver.find_element_by_id("mi0,7").click()#退保
        elif dataDict['異動別']=="3":
            driver.find_element_by_id("mi0,8").click()#調保
        time.sleep(0.8)

        if "健保投保單位代碼" in dataDict:
            driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
        else:
            driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

        driver.find_element_by_id("uploadFile").send_keys(excelFilePath)
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)
    #     time.sleep(3)
        # driver.find_element_by_id("btnSaveB").click()
        driver.find_element_by_id("mi0,53").click()
        time.sleep(1)

    driver.close()
except Exception as e:
    msg = EmailMessage()
    msg.set_content("error: "+str(e))
    msg['Subject'] = 'surcure'
    msg['From'] = "lisa"
    msg['To'] = 'andy.yuan@wowprime.com'
    with smtplib.SMTP('192.168.2.1',25) as s:
        s.send_message(msg)

KeyboardInterrupt: 

In [107]:

#     {'個人自願提繳率(%)': '',
#  '勞基法特殊身分別': '',
#  '勞工保險證號': '05952202',
#  '勞工保險證號檢查碼': 'F',
#  '勞退提繳日期(與勞保加保日期不同時才需輸入)': '',
#  '已領取社會保險給付種類': '',
#  '提繳身分別': '1',
#  '月實際工資': '11100',
#  '格式別': '1',
#  '特殊身分別': '1',
#  '異動別': '4',
#  '被保險人出生日期': '0880224',
#  '被保險人外籍': '',
#  '被保險人姓名': '吳名豪',
#  '被保險人性別': '',
#  '被保險人身份證號': 'A130528512',
#  '雇主提繳率(%)': '6'}

# 最新進度

In [26]:
from tkinter import Tk,StringVar,Label,Entry,Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait#直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import sys
import time 
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage

b=time.time()

uNoList=set()
report=[]
changDict={"4":"加保","2":"退保","3":"調保"}
web='https://edesk.bli.gov.tw/cpa/'


# def mailTo(title,mailAdd,message,whoSend='AutoAddInsurance'):
#     msg = EmailMessage()
#     msg.set_content(message)
#     msg['Subject'] = title
#     msg['From'] = whoSend
#     msg['To'] = mailAdd
#     with smtplib.SMTP('192.168.2.1',25) as s:
#         s.send_message(msg)
    
with open(r'D:\保險申報檔案\試打包\Email.txt') as f:
    mails=f.read().split('\n')
while "" in mails:
    mails.remove("")

def mailTo(title,mailAdds,message,whoSend='AutoAddInsurance'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()        
    
    


def doAddPopChangeAssure():
    try:
        errorMsg=""
        pinNum=e2.get() #pin號碼
        chrome_options = Options()
        #crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
        chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
        
        chrome_options.add_experimental_option("prefs", {
          "download.default_directory": e3.get(),
          "download.prompt_for_download": False,
          "download.directory_upgrade": True,
          "safebrowsing.enabled": True
        })
        
        #webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
        #套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
        driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)
        
        dataDir=e1.get() #需改成人工輸入
        #遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
        # for i in os.walk(dataDir):
        #     for j in i[2]:
        #         print(i[0]+j)
        #可縮寫成這樣     
        pathList=[i[0]+"/"+j for i in os.walk(dataDir) for j in i[2]]
        myxlobject= xl2dict.XlToDict()
        for excelFilePath in pathList:
            #得到excel內容成dict
            dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
            #進入勞保局
            WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
            time.sleep(0.2)
        #     alert=driver.switch_to_alert()
        #     alert.dismiss()
            #關閉阻擋頁
            driver.find_element_by_class_name("b-close").click()

            #輸入pin碼(自然人憑證)
            pinInput=driver.find_element_by_name("pin")
            pinInput.clear()
            pinInput.send_keys(pinNum)#憑證pin碼
            #輸入保險證號
            unoInput=driver.find_element_by_name("uno")
            unoInput.clear()
            unoNumber=dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']#"單位保險證號"
            unoInput.send_keys(unoNumber)
            uNoList.add(unoNumber)
            time.sleep(0.5)
            #確認進入
            driver.find_element_by_name('Submit').click()
            time.sleep(2)
            try:
                WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
                time.sleep(0.2)
                driver.find_element_by_class_name("b-close").click()
            except:
                report.append("登入失敗-異動別:{}\t證號:{}\t位置{}".
                                  format(changDict[dataDict['異動別']],
                                         dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                         excelFilePath))
                time.sleep(0.3)
                alert=driver.switch_to_alert()
                ###############################################需增加判斷密碼錯誤、權限錯誤問題
                if '讀卡機' in alert.text:
                    errorMsg="讀卡機沒插卡"
#                     mailTo('ReportError','andy.yuan@wowprime.com',errorMsg)
                    mailTo('ReportError',mails,errorMsg)
                    driver.quit()
                    root.destroy()
                    sys.exit("RRR")
                elif '權限' in alert.text:
                    errorMsg="你沒權限跑{}".format(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
#                     mailTo('ReportError','andy.yuan@wowprime.com',errorMsg)
                    mailTo('ReportError',mails,errorMsg)
                    driver.quit()
                    root.destroy()
                    sys.exit("RRR")
                elif '密碼' in alert.text:
                    errorMsg="密碼錯誤>>你所輸入的密碼是:{}".format(pinNum)
#                     mailTo('ReportError','andy.yuan@wowprime.com',errorMsg)
                    mailTo('ReportError',mails,errorMsg)
                    driver.quit()
                    root.destroy()
                    sys.exit("RRR")
                ###############################################
                alert.accept()
                continue
            
            WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.ID,'mi0,0')))
            time.sleep(0.5)
            driver.find_element_by_id("mi0,0").click()
            time.sleep(0.2)
            driver.find_element_by_id("mi0,5").click()
            WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.ID,'mi0,7')))
            time.sleep(0.2)

            if dataDict['異動別']=="4":
                driver.find_element_by_id("mi0,6").click()#加保
            elif dataDict['異動別']=="2":
                driver.find_element_by_id("mi0,7").click()#退保
            elif dataDict['異動別']=="3":
                driver.find_element_by_id("mi0,8").click()#調保
            time.sleep(1)

            if "健保投保單位代碼" in dataDict:
                driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
            else:
                driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

            driver.find_element_by_id("uploadFile").send_keys(excelFilePath)
            time.sleep(0.5)
            driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)
            time.sleep(0.2)
            driver.find_element_by_id("btnSaveB").click()
            #睡到成功為止或過了15秒
            def success():
                try:
                    return '資料新增成功' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text
                except:
                    return False
            nsec=0
            while (not success()):
                try:
                    if '失敗' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text:
                        report.append("上傳失敗-異動別:{}\t證號:{}\t位置{}".
                                      format(changDict[dataDict['異動別']],
                                             dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                             excelFilePath))
                        driver.find_element_by_id("mi0,53").click()
                        continue
                except:
                    pass
                nsec+=0.5
                if nsec>15:
                    break
                time.sleep(0.5)
#                 print(1)
            if success():
#                 report.append("資料上傳成功-異動別:{}\t證號:{}\t位置{}".
#                                   format(changDict[dataDict['異動別']],
#                                          dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
#                                          excelFilePath))
                driver.find_element_by_id("mi0,53").click()
            time.sleep(1)
        
        
        #########################################################下載區!!!
        for uNo in uNoList:
            try:
                WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
                time.sleep(0.2)
                #關閉阻擋頁
                driver.find_element_by_class_name("b-close").click()
                #輸入pin碼(自然人憑證)
                time.sleep(0.1)
                pinInput=driver.find_element_by_name("pin")
                pinInput.clear()
                pinInput.send_keys(pinNum)#憑證pin碼
                #輸入保險證號
                time.sleep(0.1)
                unoInput=driver.find_element_by_name("uno")
                unoInput.clear()
                unoInput.send_keys(uNo)#"單位保險證號"
                time.sleep(0.5)
                #確認進入
                driver.find_element_by_name('Submit').click()
                time.sleep(2)
                try:
                    WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
                    time.sleep(0.2)
                    driver.find_element_by_class_name("b-close").click()
                except:
                    report.append("下載時登入失敗-證號:{}".format(uNo))
                    time.sleep(0.2)
                    alert=driver.switch_to_alert()
                    alert.accept()
                    continue
                WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.ID,'mi0,0')))
                time.sleep(0.5)
                driver.find_element_by_id("mi0,0").click()
                time.sleep(0.1)
                driver.find_element_by_id("mi0,10").click()
                time.sleep(0.2)
                driver.find_element_by_xpath('//*[@id="main"]/form/fieldset/table/tbody/tr[7]/td[2]/input[2]').click()
                time.sleep(0.1)
                driver.find_element_by_name('btnQuery').click()
                time.sleep(0.5)

                downloadList=['https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchPayList',
                'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchStopList',
                'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchAdjList']

                for website in downloadList:
                    driver.get(website)
                    time.sleep(0.1)
                    WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.NAME,'btnBackQueryForm')))
                    time.sleep(0.2)
                    if len(driver.find_elements_by_name('meslamIdOfPrint'))>0:
                        for point in driver.find_elements_by_name('meslamIdOfPrint'):
                            point.click()
                            time.sleep(0.1)
                            driver.find_element_by_id('btnPrint2').click()
                            time.sleep(0.5)
                        time.sleep(0.5)
                        driver.back()
                        time.sleep(0.4)
#                 report.append("下載成功-證號:{}".format(uNo))
                driver.find_element_by_id("mi0,53").click()
                time.sleep(1)
            except Exception as e:
                time.sleep(0.5)
                report.append("下載失敗-證號:{}".format(uNo))
                driver.find_element_by_id("mi0,53").click()
                time.sleep(1)
        #########################################################更名區
        path=e3.get()
        
#         pathList2=[i[0]+"/"+j for i in os.walk(path) for j in i[2]]
        for pathTuple in os.walk(path):
            for file_ in pathTuple[2]:
                

#         for file_ in os.listdir(path):
                name=[]
#                 book = xlrd.open_workbook(os.path.join(path, file_))
                book = xlrd.open_workbook(os.path.join(pathTuple[0], file_))
                sheet=book.sheet_by_index(0)

                #日期
                dateCell=sheet.cell_value(rowx=3,colx=8) or sheet.cell_value(rowx=3,colx=4)
                if len(dateCell.split("／"))<3:
                    name.append(dateCell.split("：")[1].split(" ")[0])
                else:
                    name.append(dateCell.split("／")[2] or dateCell.split("／")[1].split("：")[1].split(" ")[0])

                ##事業部
                if "公司" in sheet.cell_value(rowx=4,colx=0):
                    name.append(sheet.cell_value(rowx=4,colx=0).split("公司")[1] or sheet.cell_value(rowx=3,colx=0).split("：")[1])
                else:
                    name.append(sheet.cell_value(rowx=3,colx=0).split("公司")[1] or sheet.cell_value(rowx=2,colx=0).split("：")[1])


                ##加退調保
                if '加' in sheet.cell_value(rowx=0,colx=0):
                    name.append("加")
                elif '退' in sheet.cell_value(rowx=0,colx=0):
                    name.append("退")
                elif '調' in sheet.cell_value(rowx=0,colx=0):
                    name.append("調")

                ##二、三合一
                if '健' in sheet.cell_value(rowx=1,colx=0):
                    name.append("3")
                else:
                    name.append("2")

                try:
                    os.rename(os.path.join(pathTuple[0],file_),os.path.join(pathTuple[0],"_".join(name)+".xls"))
                except:
                    print(os.path.join(path, file_)+"\t已有變更檔名的檔案")
        
        #########################################################
        
        driver.quit()
        print("\n".join(report))
        
        e=time.time()
        costTime=round(e-b)
#         mailTo('ReportForSuccessAndFail',
#                'andy.yuan@wowprime.com',
#                web+"\n"+"\n".join(report)+"\n花費時間:{}秒\n共處理{}筆資料".format(costTime,len(pathList)))
        mailTo('ReportForSuccessAndFail',
               mails,
               web+"\n"+"\n".join(report)+"\n花費時間:{}秒\n共處理{}筆資料".format(costTime,len(pathList)))
        
#         msg = EmailMessage()
#         msg.set_content(web+"\n"+"\n".join(report))
#         msg['Subject'] = 'ReportForSuccessAndFail'
#         msg['From'] = "AutoAddInsurance"
#         msg['To'] = 'andy.yuan@wowprime.com'
#         with smtplib.SMTP('192.168.2.1',25) as s:
#             s.send_message(msg)



    except Exception as e:
        if errorMsg=="":
            errorMsg=str(e)+'\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com'
        
        print('出錯了RRRRRRRRRRRR'+str(e)+'\n')
        driver.quit()
        
#         mailTo('ReportError','andy.yuan@wowprime.com',errorMsg)
        mailTo('ReportError',mails,errorMsg)
        
#         msg = EmailMessage()
#         msg.set_content(errorMsg)
#         msg['Subject'] = 'ReportError'
#         msg['From'] = "AutoAddInsurance"
#         msg['To'] = 'andy.yuan@wowprime.com'
#         with smtplib.SMTP('192.168.2.1',25) as s:
#             s.send_message(msg)

def selectPath():
    path_ = askdirectory()
    path.set(path_)
def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)
root = Tk()
path = StringVar()
Label(root,text = "讀取資料夾:").grid(row = 0, column = 0)
e1=Entry(root, textvariable = path)
e1.grid(row = 0, column = 1)
Button(root, text = "讀取資料夾", command = selectPath).grid(row = 0, column = 2)

path2 = StringVar()
Label(root,text = "存放資料夾:").grid(row = 1, column = 0)
e3=Entry(root, textvariable = path2)
e3.grid(row = 1, column = 1)
Button(root, text = "存放資料夾", command = selectPath2).grid(row = 1, column = 2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column = 0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='加退調保開始', command=doAddPopChangeAssure).grid(row=3, column=1, pady=4)
root.mainloop()
    
    



D:/保險申報檔案/存放區\CaApplyBatchData_1070209 (3).xls	已有變更檔名的檔案
登入失敗-異動別:調保	證號:05952199DDD	位置D:/保險申報檔案/test/保證改掉sample.xls
上傳失敗-異動別:退保	證號:05952199X	位置D:/保險申報檔案/test/投保號碼改掉sample.xls
上傳失敗-異動別:加保	證號:05952199X	位置D:/保險申報檔案/test/改投保.xls
上傳失敗-異動別:調保	證號:05952199X	位置D:/保險申報檔案/test/改第二保險證號.xls
下載時登入失敗-證號:05952199DDD


# 一步一步測試區

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage
import xlrd

pinNum="780311" #需改成人工輸入

chrome_options = Options()
#crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
#webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
#套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
driver.get("https://edesk.bli.gov.tw/cpa/")

#找到excel資料存放的位置，將所有檔案的路徑變成list
dataDir=r"D:\保險申報檔案\保險申報檔案\11802" #需改成人工輸入
#遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
# for i in os.walk(dataDir):
#     for j in i[2]:
#         print(i[0]+j)
#可縮寫成這樣     
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
myxlobject= xl2dict.XlToDict()
excelFilePath=pathList[0]
#得到excel內容成dict
dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
#進入勞保局
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
#     alert=driver.switch_to_alert()
#     alert.dismiss()
#關閉阻擋頁
driver.find_element_by_class_name("b-close").click()
#輸入pin碼(自然人憑證)
pinInput=driver.find_element_by_name("pin")
pinInput.clear()
pinInput.send_keys(pinNum)#憑證pin碼
#輸入保險證號
unoInput=driver.find_element_by_name("uno")
unoInput.clear()

print(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']+dataDict['異動別'])
unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
uNoList=set()
uNoList.add(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
time.sleep(0.5)
#確認進入
driver.find_element_by_name('Submit').click()
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
passOrNot=True
driver.find_element_by_class_name("b-close").click()
time.sleep(0.4)
driver.find_element_by_id("mi0,0").click()
time.sleep(0.1)
driver.find_element_by_id("mi0,5").click()
time.sleep(0.2)

print(dataDict['異動別'])
if dataDict['異動別']=="4":
    driver.find_element_by_id("mi0,6").click()#加保
elif dataDict['異動別']=="2":
    driver.find_element_by_id("mi0,7").click()#退保
elif dataDict['異動別']=="3":
    driver.find_element_by_id("mi0,8").click()#調保
time.sleep(0.8)

if "健保投保單位代碼" in dataDict:
    driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
else:
    driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

driver.find_element_by_id("uploadFile").send_keys(excelFilePath)
time.sleep(0.5)
driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)
#     time.sleep(3)
driver.find_element_by_id("btnSaveB").click()

def success():
    try:
        return '資料新增成功' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text
    except:
        return False
while not success():
    time.sleep(0.5)


05952199X4


UnexpectedAlertPresentException: Alert Text: None
Message: unexpected alert open: {Alert text : 讀卡機沒插卡}
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)


In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage
import xlrd

pinNum="780311" #需改成人工輸入

chrome_options = Options()
#crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
#webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
#套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
driver.get("https://edesk.bli.gov.tw/cpa/")

#找到excel資料存放的位置，將所有檔案的路徑變成list
dataDir=r"D:\保險申報檔案\保險申報檔案\11802" #需改成人工輸入
#遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
# for i in os.walk(dataDir):
#     for j in i[2]:
#         print(i[0]+j)
#可縮寫成這樣     
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
myxlobject= xl2dict.XlToDict()
excelFilePath=pathList[0]
#得到excel內容成dict
dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
#進入勞保局
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
#     alert=driver.switch_to_alert()
#     alert.dismiss()
#關閉阻擋頁
time.sleep(3)
driver.find_element_by_class_name("b-close").click()
#輸入pin碼(自然人憑證)
pinInput=driver.find_element_by_name("pin")
pinInput.clear()
pinInput.send_keys(pinNum)#憑證pin碼
#輸入保險證號
unoInput=driver.find_element_by_name("uno")
unoInput.clear()

print(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']+dataDict['異動別'])
unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
uNoList=set()
uNoList.add(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
time.sleep(0.5)
#確認進入
driver.find_element_by_name('Submit').click()
time.sleep(0.5)


05952199X4


In [9]:
import sys

errorMsg=""
report=[]
try:
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
        time.sleep(0.2)
        driver.find_element_by_class_name("b-close").click()
    except:
        report.append("登入失敗-異動別:{}\t證號:{}\t位置{}")
        time.sleep(0.3)
        alert=driver.switch_to_alert()
        ###############################################需增加判斷密碼錯誤、權限錯誤問題
        if '讀卡機' in alert.text:
            errorMsg="讀卡機沒插卡"
            driver.quit()
            sys.exit("RRR")
        elif '權限' in alert.text:
            errorMsg="你沒權限跑{}".format(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
            driver.quit()
            sys.exit("RRR")
        elif '密碼' in alert.text:
            errorMsg="你密碼錯誤"
            driver.quit()
            sys.exit("RRR")

        print('safsfadfas')

        ###############################################
    print(errorMsg+"aaa")
except:
    print(errorMsg)
    
# try:
#     alert=driver.switch_to_alert()
    
# except:
#     print('xxxxx')

讀卡機沒插卡


In [6]:

# report.append("登入失敗-異動別:{}\t證號:{}\t位置{}")
# time.sleep(0.3)
# alert=driver.switch_to_alert()
# ###############################################需增加判斷密碼錯誤、權限錯誤問題
# if '讀卡機' in alert.text:
#     errorMsg="讀卡機沒插卡"
#     driver.quit()
#     os._exit(1)
# elif '權限' in alert.text:
#     errorMsg="你沒權限跑{}".format(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
#     driver.quit()
#     os._exit(1)
# elif '密碼' in alert.text:
#     errorMsg="你密碼錯誤"
#     driver.quit()
#     os._exit(1)

# print('safsfadfas')

SystemExit: RRR

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [59]:
uNoList=set()
uNoList.update(['05952202F','05952199X'])

for uNo in uNoList:
    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
    time.sleep(0.2)
    #關閉阻擋頁
    driver.find_element_by_class_name("b-close").click()
    #輸入pin碼(自然人憑證)
    time.sleep(0.1)
    pinInput=driver.find_element_by_name("pin")
    pinInput.clear()
    pinInput.send_keys(pinNum)#憑證pin碼
    #輸入保險證號
    time.sleep(0.1)
    unoInput=driver.find_element_by_name("uno")
    unoInput.clear()
    unoInput.send_keys(uNo)#"單位保險證號"
    time.sleep(0.2)
    #確認進入
    driver.find_element_by_name('Submit').click()
    time.sleep(2)
    try:
        WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
        time.sleep(0.2)
        driver.find_element_by_class_name("b-close").click()
    except:
        report.append("下載資料登入失敗-證號:{}".format(uNo,))
        time.sleep(0.2)
        alert=driver.switch_to_alert()
        alert.accept()
        continue
    time.sleep(0.3)
    driver.find_element_by_id("mi0,0").click()
    time.sleep(0.2)
    driver.find_element_by_id("mi0,10").click()
    time.sleep(0.2)
    driver.find_element_by_xpath('//*[@id="main"]/form/fieldset/table/tbody/tr[7]/td[2]/input[2]').click()
    time.sleep(0.1)
    driver.find_element_by_name('btnQuery').click()
    time.sleep(0.5)

    downloadList=['https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchPayList',
    'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchStopList',
    'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchAdjList']

    for website in downloadList:
        driver.get(website)
        time.sleep(0.5)
        if len(driver.find_elements_by_name('meslamIdOfPrint'))>0:
            for point in driver.find_elements_by_name('meslamIdOfPrint'):
                point.click()
                time.sleep(0.1)
                driver.find_element_by_id('btnPrint2').click()
                time.sleep(0.5)
            time.sleep(0.5)
            driver.back()
            time.sleep(0.4)
    driver.find_element_by_id("mi0,53").click()
    time.sleep(1)

NameError: name 'driver' is not defined

In [62]:
import xlrd
book = xlrd.open_workbook(r'D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208.xls')
sheel_1 = book.sheet_by_index(0)

In [63]:
book.sheet_by_name("本人").cell_value(rowx=6,colx=3)

'吳名豪'

In [19]:
#頁面名稱
print("Worksheet name(s): ",book.sheet_names()[0])

Worksheet name(s):  本人


In [20]:
#有幾頁
print('book.nsheets',book.nsheets)

book.nsheets 1


In [21]:
#使用了幾行幾列
print('sheel_1.name:',sheel_1.name,'sheel_1.nrows:',sheel_1.nrows,'sheel_1.ncols:',sheel_1.ncols)

sheel_1.name: 本人 sheel_1.nrows: 8 sheel_1.ncols: 12


In [24]:
#查表格內容 rowx,colx由0開始
print(sheel_1.cell_value(rowx=6,colx=3))

A1: 居留證/護照號碼


In [ ]:
workbook = xlwt.Workbook()
sheet = workbook.add_sheet('test')

for index, value in enumerate(data):
    sheet.write(0, index, value)

workbook.save('output.xls')




In [8]:
import xlrd
path=r"D:\保險申報檔案\存放區\sample"
for file_ in os.listdir(path):
    name=[]
    book = xlrd.open_workbook(os.path.join(path, file_))
    sheet=book.sheet_by_index(0)
    
    #日期
    dateCell=sheet.cell_value(rowx=3,colx=8) or sheet.cell_value(rowx=3,colx=4)
    if len(dateCell.split("／"))<3:
        name.append(dateCell.split("：")[1].split(" ")[0])
    else:
        name.append(dateCell.split("／")[2] or dateCell.split("／")[1].split("：")[1].split(" ")[0])

    ##事業部
    if "公司" in sheet.cell_value(rowx=4,colx=0):
        name.append(sheet.cell_value(rowx=4,colx=0).split("公司")[1] or sheet.cell_value(rowx=3,colx=0).split("：")[1])
    else:
        name.append(sheet.cell_value(rowx=3,colx=0).split("公司")[1] or sheet.cell_value(rowx=2,colx=0).split("：")[1])
    
    
    ##加退調保
    if '加' in sheet.cell_value(rowx=0,colx=0):
        name.append("加")
    elif '退' in sheet.cell_value(rowx=0,colx=0):
        name.append("退")
    elif '調' in sheet.cell_value(rowx=0,colx=0):
        name.append("調")
        
    ##二、三合一
    if '健' in sheet.cell_value(rowx=1,colx=0):
        name.append("3")
    else:
        name.append("2")
    
    try:
        os.rename(os.path.join(path,file_),os.path.join(path,"_".join(name)+".xls"))
    except:
        print(os.path.join(path, file_)+"\t已有變更檔名的檔案")

D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (3).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (6).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (7).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (8).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (9).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208.xls	已有變更檔名的檔案


In [49]:
sheetttt=book.sheet_by_name("本人")

In [77]:
x=["",'aaa']
for i in x:
    print(i or "xx")

xx
aaa


In [ ]:
os.rename(os.path.join(path,file_),os.path.join(path,"_".join(name)+".xls"))

In [87]:
"aaa".split("＼")

['aaa']